In [43]:
import os
import datetime
import gc
import glob
import geopandas
import random
from sentinelsat import SentinelAPI, geojson_to_wkt, read_geojson

class sentinel1_download_preprocess():
    def __init__(self,username,password,date_1,date_2,lat,lon,query_style,footprint,download):
        self.username = username
        self.password = password
        self.date_start = datetime.datetime.strptime(date_1, "%d/%m/%y")
        self.date_end = datetime.datetime.strptime(date_2, "%d/%m/%y")
        self.lat = lat
        self.lon = lon
        self.query_style = query_style
        self.footprint = geojson_to_wkt(read_geojson(footprint))
        self.download = download
            # configurations
        self.api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
        self.platformname ='Sentinel-2'
        self.orbitdirection = 'DESCENDING'  # ASCENDING, DESCENDING
        self.processinglevel = 'Level-2A'
        self.cloudcoverpercentage = (0,30)
        
    @classmethod
    def from_input(cls):
        return cls(
            input('username:'),
            input('password:'),
            input('date_1:'),
            input('date_2:'),
            input('lat:'),
            input('lon:'),
            input('query_style:'),
            input('footprint:'),
            input('download:'),
        )  


    def sentinel1_download(self):
        global download_candidate
        if self.query_style == 'coordinate':
            download_candidate = self.api.query('POINT({0} {1})'.format(self.lon, self.lat),
                                                date=(self.date_start, self.date_end),
                                                platformname=self.platformname,
                                                orbitdirection=self.orbitdirection,
                                                processinglevel=self.processinglevel,
                                                cloudcoverpercentage=self.cloudcoverpercentage)
        elif self.query_style == 'footprint':
            download_candidate = self.api.query(self.footprint,
                                                 date=(self.date_start, self.date_end),
                                                platformname=self.platformname,
                                                orbitdirection=self.orbitdirection,
                                                processinglevel=self.processinglevel,
                                                cloudcoverpercentage=self.cloudcoverpercentage)


            
            dataproduct = self.api.to_geodataframe(download_candidate)
            d = []
            for i in range(len(dataproduct.uuid)):
                product_id = self.api.get_product_odata(dataproduct['uuid'][i])
               # print(f"show this {product_id}")
                for j in product_id.keys():
                    if (j == 'Online' and product_id[j] == True):
                        d.append(dataproduct['uuid'][i])

            print(f"len of the list = {len(d)}")
            self.api.download(random.choice(d))
        else:
            print("Define query attribute")
                                   
                            
                        


In [ ]:
user = sentinel1_download_preprocess.from_input()

In [ ]:
user.sentinel1_download()